In [1]:
import pandas as pd
from pyhive import hive

def execute_query(cursor, query, fetch=False):
    try:
        cursor.execute(query)
        if fetch:
            return cursor.fetchall()
    except Exception as e:
        print(e)
        return e

conn = hive.Connection(host="localhost", port=10000, username="hive")
cursor = conn.cursor()

execute_query(cursor, '''
SHOW DATABASES
''', True)

[('default',)]